## Streaming Practical 1

In [32]:
import findspark
findspark.init()
import pyspark

### Streaming from TCP Socket


In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
TCP = (spark.readStream.format('socket')
         .option('host','localhost')
         .option('port',9090)
         .load()
        )

### Print Schema

In [35]:
TCP.printSchema()

root
 |-- value: string (nullable = true)



In [36]:
TCP.isStreaming

True

### After processing, you can write the DataFrame to console.

In [37]:
from pyspark.sql.functions import explode, split
df_words=TCP.select(explode(split(TCP.value,' ')).alias('word'))

In [38]:
count=df_words.groupBy("word").count()

In [40]:
count.writeStream\
      .format("console")\
      .outputMode("complete")\
      .start()\
      .awaitTermination(120)


21/10/07 01:40:31 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1f815e21-9e0c-4828-a3ee-ffdb293b7a8f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
| Trying|    2|
|     HW|    2|
|    you|    1|
|example|    2|
|  HelLo|    2|
|     Hi|    1|
|  doing|    1|
|    How|    1|
|    TCP|    2|
|    are|    1|
+-------+-----+



False

21/10/07 01:42:51 WARN TextSocketMicroBatchStream: Stream closed by localhost:9090


## Task 2

let’s create a streaming DataFrame that represents text data received from a server listening on localhost:9999, and transform the DataFrame to calculate word counts.

### Create DataFrame representing the stream of input lines from connection to localhost:9999


In [41]:
df = (spark.readStream.format('socket')
         .option('host','localhost')
         .option('port',9999)
         .load()
        )

21/10/07 01:43:54 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


### Split the lines into words


In [42]:
from pyspark.sql.functions import explode, split
words = df.select(explode(split(df.value,' ')).alias('word'))

### Generate running word count


In [43]:
wordCounts = words.groupBy("word").count()

### Start running the query that prints the running counts to the console


In [45]:
checkpointDir = "chkpont"
streamingQuery = (wordCounts.writeStream
                  .format("console")
                  .outputMode("complete")
                  .trigger(processingTime="1 second")
                  .option("checkpointLocation", checkpointDir)
                  .start())
streamingQuery.awaitTermination(120)

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



21/10/07 01:45:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 63810 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|  name|    2|
|asking|    2|
|   you|    6|
|thanks|    2|
|   for|    2|
|    Hi|    2|
|    is|    4|
|   All|    2|
|  Fine|    2|
| doing|    2|
|   How|    2|
|    my|    2|
| thank|    2|
|  what|    2|
|   are|    2|
|  well|    2|
| Reham|    2|
| about|    2|
+------+-----+



21/10/07 01:46:15 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 52108 milliseconds


False